# Introduction/Project Overview:

In this notebook, I will go over the World Health Organization Life Expectancy dataset. This dataset can be found on [kaggle](https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who). We are given data on a country, population, health issues, life expectancy, etc. Our goal would be to create a model that can accurately determine the life expectency given a some characteristics on a population. Throughout this notebook I will visualize the data, explain some data preprocessing techniques, construct and evaluate models and analyze the results.

### Data Exploration & Preprocessing:
I will go over the dataset, analyzing its various features, checking for missing values, and gaining insights into the distribution of variables. Prior to building the models, I will preprocess the data by handling missing values, encoding categorical variables, and scaling numerical features to ensure good model performance.

### Model Building & Evaluation:
In this notebook I will try implement a couple of models to try and see which ones are more accurate at predicting life expectancy. This is a supervised learning task since we are given the life expectancy of these population. Additionally this is a regression tasks because we are trying to predict a numerical value rather then put something into a category. For this notebook the models I chose is multiple linear regression, polynomial regression and a neural network. Lastly I will interpret the results of each model. 


### Conclusion:
Finally, I will be discussing potential areas for model improvement, what stood out to me and what were some challanges. The conclusion serves more as a reflection for me on my time working on this notebook. This will serve as a good test for me to keep learning and testing my skills. Lets get to it!

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

# Data Exploration & Preprocessing:
As mentioned earlier I got the dataset from kaggle. The link to that can be found above in the project overview. The download came with a csv file. Since I have it locally on my computer I can eassily access the data as shown below. Some of the first steps we will do before creating a model is to see what our data looks like.

In [ ]:
data = pd.read_csv('./Life Expectancy Data.csv')

Lets take a look at the first couple of entries in our dataset and what columns are int it.

In [ ]:
data.head(5)

In [ ]:
data.info()

So we have 22 columns in total. We have three categorical columns and 19 numeric columns. We also have quite some missing values so we will try and fill them later. The column names are pretty straight forward but if you would like to read more on them there are explanations on kaggle.

Lets see what other information we can get at a glance from our data. 

In [ ]:
data.describe()

From the table we can see that the minimum life expectancy is 36 while the max is 89. The max adult mortality is 723 per thousand in population, while the min is 1. The average alcohol consumption per liters per capita is a lot lower than at I expected but the min is 0.01 which makes sense as some countries might have religious restrictions. It is also very disturbing seeing the low immunization coverage for some of the diseases. Especially when the average is pretty high for some. Overall this table gives us some interesting information on the distribution of the data. 

Below I am changing the column names because they previously gave me issues the way it was originally formated. 

In [ ]:
data.columns = [x.lower().strip().replace(' ', '_') for x in data.columns]

Now we are selecting the numerical and categorical columns so we can do some light data exploration.

In [ ]:
numerical = data.select_dtypes(include=['float64', 'int64']).columns

In [ ]:
categorical = data.select_dtypes(include=['object']).columns

Lets start by checking the differences between developed and developing countries.

In [ ]:
pd.pivot_table(data, index='status', values=numerical)

Above we can see that there are very big differences between the two types of countries. Developing countires have higher moratlity rates, and overall haelth issues. While the difference in immunization coverage for developed and developing countries is not a huge difference, developing countries still have less of it, which is important to note. Now lets take a look at the same data by each country.

In [ ]:
pd.pivot_table(data, index='country', values=numerical)

I realized that there are way too many countries to do some analysis on. I did not know that there was this many in the dataset. Regardless we can see that all these countries have distinct values. It would be interesting to see if we could possibly plot this. 

Now we are going to be ploting life expectancy against all other numerical columns to see if there is a relationship. We are using life expectancy because this is the column of interest that we are trying to predict later.

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(20, 15))
axes = axes.flatten()
for i, col in enumerate(numerical):
    axes[i].scatter(data[col], data['life_expectancy'])  
    axes[i].set_title(f'{col} vs life_expectancy')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('life_expectancy')
plt.tight_layout()
plt.show()

From the graphs above there does not seem to be much of a relationship between life expectancy and the other columns in the dataset. By that I mean that there is no very noticable relationship.  The columns that seem to have some relationship are adult mortality (makes sense), infant deaths, percentage expenditure, under five deaths, hiv/aids, and income composition of resources. Of these adult mortality and income composition of resources seem to be the most coralated. It is also interesting to see infant deaths and under five deaths seem very similar. Same for the thiness columns.

Lastly there seems to be a group of populations that are constant in their life expectancy. For example if we look at income composition of resources we see that even at 0.0 the life expectancy changes up and down. This pattern can be seen through most graphs above. It might be some inconsistent or false reporting. 

Now we move on to graphing the distribution of the columns. 

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(20, 15))  
axes = axes.flatten()
for i, col in enumerate(numerical):
    axes[i].hist(data[col])
    axes[i].set_title(col)
plt.tight_layout()
plt.show()

Everything seems alright, theres no big outliers. We will normalize and scale the data later. Lets now count the number of developed and devloping countries. 

In [ ]:
data['status'].value_counts() 

## Filling missing Values
Now that we have taken a look at our data, checked patterns, etc lets see what we can do about the missing values. First we must count how many values we are missing and where. 

In [ ]:
data.isnull().sum()

We have a lot of missing values all through out the columns. Most missing value is population, hepatitis and gdp. We can look into some patterns of which countries have missing values and try to see why. We can then start looking into countries that are similar to fill in the values. We'll start by seperating the data that has missing values.

In [ ]:
missing_data = data[data.isnull().any(axis=1)]

In [ ]:
missing_data.head(5)

In [ ]:
missing_data.country.value_counts()

The cell above tells us which countries seem to appear in the missing dataframe more often. We can use this to see if its a reocurring issue or a one time thing. If these countries appear often or if their values are missing often then we could look into it.

In [ ]:
missing_data.status.value_counts()

For now lets look at other interesting details. There seems to be more developing countires with missing values whihc makes sense because they might not have the infrustructure to keep count of all the data. 

Now we want to check the rows and see the ones that only have 1 missing value.

In [ ]:
missing_counts = missing_data.isnull().sum(axis=1)
df_one_missing = missing_data[missing_counts == 1]

In [ ]:
# missing_data[(missing_data['country'] == 'female') & (data['Pclass'] == 1) & (data['Fare'] >= 80) &

In [ ]:
len(df_one_missing)

In [ ]:
df_one_missing.isnull().sum()